<h1>Book Contents</h1>
<li>Creation of 2D Integer-to-Integer 5/3 Lifting Wavelet transform</li>
<li>Perform Transform on a file (irrespective of format, <a href=https://ieeexplore.ieee.org/document/8692644>HanQiu[2021]</a> ) at level 2</li>


In [ ]:
from PIL import Image
from IPython.display import display, HTML

def display_images_in_grid(ll, hl, lh, hh):
    
    display(HTML('<h2>LL</h2>'))
    display(Image.fromarray(ll))
    
    display(HTML('<h2>HL</h2>'))
    display(Image.fromarray(hl))

    display(HTML('<h2>LH</h2>'))
    display(Image.fromarray(lh))

    display(HTML('<h2>HH</h2>'))
    display(Image.fromarray(hh))

<h1>2D DWT Transform.</h1>

In [ ]:
import numpy as np

def dwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len

    for i in range(1, detail_len):
        seq[(i << 1) - 1] -= (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] -= (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    seq[0] += (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] += (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] += (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    return seq

def idwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len
    
    seq[0] -= (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] -= (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] -= (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    for i in range(1, detail_len):
        seq[(i << 1) - 1] += (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] += (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    return seq


def dwt2d(seq):
    
    horizontal_transform = np.apply_along_axis(dwt, axis=1, arr=seq)
    l = horizontal_transform[:,::2]
    h = horizontal_transform[:,1::2]
      
    vertical_transform_l = np.apply_along_axis(dwt, axis=0, arr=l)
    ll = np.array(vertical_transform_l[::2,:])
    lh = np.array(vertical_transform_l[1::2,:])
 
    vertical_transform_h = np.apply_along_axis(dwt, axis=0, arr=h)
    hl = np.array(vertical_transform_h[::2,:])    
    hh = np.array(vertical_transform_h[1::2,:])
    
    return ll, hl, lh, hh


def idwt2d(ll, hl, lh, hh):
    
    l_rows = ll.shape[0] + lh.shape[0]
    l_cols = max(ll.shape[1], lh.shape[1])
    vertical_transform_l = np.empty((l_rows, l_cols), dtype=ll.dtype)
    vertical_transform_l[::2, :] = ll
    vertical_transform_l[1::2, :] = lh
    l = np.apply_along_axis(idwt, axis=0, arr=vertical_transform_l)    
    
    h_rows = hl.shape[0] + hh.shape[0]
    h_cols = max(hl.shape[1], hh.shape[1])
    vertical_transform_h = np.empty((h_rows, h_cols), dtype=hl.dtype)
    vertical_transform_h[::2, :] = hl
    vertical_transform_h[1::2, :] = hh
    h = np.apply_along_axis(idwt, axis=0, arr=vertical_transform_h)

    seq_rows = max(l.shape[0], h.shape[0])
    seq_cols = l.shape[1]+h.shape[1]
    horizontal_transform = np.empty((seq_rows, seq_cols), dtype=l.dtype)
    horizontal_transform[:, ::2] = l
    horizontal_transform[:, 1::2] = h
    seq = np.apply_along_axis(idwt, axis=1, arr=horizontal_transform)
    
    return seq
        

<br><h1>2D DWT test with image input.</h1>

In [ ]:
import cv2

def test_2d_dwt_image():

    img = cv2.imread('data/images/thumb.jpg', cv2.IMREAD_GRAYSCALE)
    ll, hl, lh, hh = dwt2d(img)
    
    display(HTML('<h2>Input Image</h2>'))
    display(Image.fromarray(img))

    display(HTML('<h2>Output 2D DWT LVL 1</h2>'))
    display_images_in_grid(ll, hl, lh, hh)

    ll2, hl2, lh2, hh2 = dwt2d(ll)
    display(HTML('<h2>Output 2D DWT LVL 2</h2>'))
    display_images_in_grid(ll2, hl2, lh2, hh2)
    
    re_seq = test_2d_idwt_image((ll, hl, lh, hh), (ll2, hl2, lh2, hh2))
    display(HTML(f'<h2>Perfect Data Reconstruction... {np.array_equal(img,re_seq)}</h2>'))
    
def test_2d_idwt_image(lvl1_coeff, lvl2_coeff):
    
    ll, hl, lh, hh = lvl1_coeff
    ll2, hl2, lh2, hh2 = lvl2_coeff
    
    ll_i = idwt2d(ll2, hl2, lh2, hh2)
    img_i = idwt2d(ll_i, hl, lh, hh)
    
    display(HTML('<h2>Reconstructed Image</h2>'))
    display(Image.fromarray(img_i))
    return img_i
    
test_2d_dwt_image()

<br><h1>Selective Encryption on a file as proposed in <a href=https://ieeexplore.ieee.org/document/8692644>HanQiu[2021]</a></h1>

In [196]:
#Util Functions
import hashlib

def bytes_to_bits(b):
    return ''.join([bin(byte)[2:].zfill(8) for byte in bytearray(b)])

def bits_to_bytes_v2(b, chunk_size=8):
    return bytes([int(''.join(b[i:i+chunk_size]), 2) for i in range(0, len(b), chunk_size)])

def sha256_with_key(key, data):
    combined = key + data
    sha256_obj = hashlib.sha256()
    sha256_obj.update(combined)
    return sha256_obj.hexdigest()

def sha512_with_key(key, data):
    combined = key + data
    sha512_obj = hashlib.sha512()
    sha512_obj.update(combined)
    return sha512_obj.hexdigest()

def xor_with_sha_key(data, sha_hex):
    sha_bytes = bytes.fromhex(sha_hex)
    xor_result = bytes([b1 ^ b2 for b1, b2 in zip(data, sha_bytes)])
    return xor_result

def recover_data_from_xor(xor_data, sha_hex):
    sha_bytes = bytes.fromhex(sha_hex)
    recovered_data = bytes([b ^ sha_bytes[i] for i, b in enumerate(xor_data)])
    return recovered_data


In [200]:
import numpy as np

file_name = 'textfile'
file_path = f'data/files/{file_name}'
buffer = 8
chunk_size = buffer * buffer
ll2_padding_length = 256
ll2_encryption_key = b'This is the key we use to encrypt the private fragment, ll2'

with open(file_path, 'rb') as file:
    while True:
        
        chunk = file.read(chunk_size)
        
        if not chunk:
            print("Reached End of the file.\n")
            break
        
        if len(chunk) < chunk_size:
            chunk += b'\x00' * (chunk_size - len(chunk))
        
        byte_array = np.frombuffer(chunk, dtype=np.uint8)
        byte_array_2d = np.reshape(byte_array, (buffer, buffer))
        
        #Step 1, 2D DWT LVL 1
        ll, hl, lh, hh = dwt2d(byte_array_2d)
        print(f'{ll.shape}, {hl.shape}, {lh.shape}, {hh.shape}')

        #Step 2, 2D DWT LVL 2
        ll2, hl2, lh2, hh2 = dwt2d(ll)
        print(f'{ll2.shape}, {hl2.shape}, {lh2.shape}, {hh2.shape}')
        
        #Step 3, SHA-256 with key, on ll2, for PPF1
        SHA256_PPF1 = sha256_with_key(ll2_encryption_key, ll2.flatten().tobytes())
        print(f'SHA256_PPF1 {SHA256_PPF1}')
        
        #Step 4, SHA-512 with key, on (lh2, hl2, hh2), for PPF2
        SHA512_PPF2 = sha512_with_key(ll2_encryption_key, np.array([lh2,hl2,hh2]).tobytes())        
        print(f'SHA512_PPF2 {SHA512_PPF2}')
        
        #Step 5, PPF1 generation, XOR using generated SHA-256 from ll2
        PPF1_bytes = np.array([lh2, hl2, hh2]).tobytes()
        PPF1_XOR = xor_with_sha_key(PPF1_bytes, SHA256_PPF1)
        print(f'PPF1_XOR {PPF1_XOR}')
        
        #Step 6, PPF2 generation, XOR using generated SHA-512 from (lh2, hl2, hh2)
        PPF2_bytes = np.array([lh, hl, hh]).tobytes()
        PPF2_XOR = xor_with_sha_key(PPF2_bytes, SHA512_PPF2)
        print(f'PPF2_XOR {PPF2_XOR}')
        
        break
print('Test Complete.')


(4, 4), (4, 4), (4, 4), (4, 4)
(2, 2), (2, 2), (2, 2), (2, 2)
SHA256_PPF1 d0f473d7f555a9a6234f02f1c5cc24c28abce952006788334c49089c6379adcd
SHA512_PPF2 eb970366c518a26ad92a3593bda7dd6a494984df4ffd3ef8383ad72c18ca2f194a6ce2c515645381683df6bc20c0297cad42db1e6f9dc20e8075884bee24e877
PPF1_XOR b'T\xa4\xe1\xa7;\x83\xf36|%W\xc2'
PPF2_XOR b'\xd4\x17\xa3<\x94\x7fQ\x92E\xc0f\xc5\xbe\xfd\xc6!Ybk%\x83\xd6\x7f\xd3\x10\x02\x1a0K."\x05\xd1\x17\xe7K\xa1l\xf8\x83\x9c\x89A\xb9\xd73\x80\xe4'
48
Test Complete.


/tmp/ipykernel_19049/2631057725.py:16: RuntimeWarning: overflow encountered in scalar add
  seq[i << 1] += (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
/tmp/ipykernel_19049/2631057725.py:17: RuntimeWarning: overflow encountered in scalar add
  seq[(approx_len - 1) << 1] += (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2
/tmp/ipykernel_19049/2631057725.py:14: RuntimeWarning: overflow encountered in scalar add
  seq[0] += (seq[1] + seq[1] + 2) >> 2
/tmp/ipykernel_19049/2631057725.py:11: RuntimeWarning: overflow encountered in scalar add
  seq[(i << 1) - 1] -= (seq[(i - 1) << 1] + seq[i << 1]) >> 1
/tmp/ipykernel_19049/2631057725.py:12: RuntimeWarning: overflow encountered in scalar add
  seq[(detail_len << 1) - 1] -= (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1
